## 使用_ _slots_ _

正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性

In [1]:
class Student(object):
    pass

In [2]:
s = Student()
s.name = 'Micheal'

In [3]:
print(s.name)

Micheal


In [5]:
def set_age(self, age):
    self.age = age

In [6]:
from types import MethodType
s.set_age = MethodType(set_age, s)
s.set_age(25)
print(s.age)

25


但是，给一个实例绑定的方法，对另一个实例是不起作用的  
为了给所有实例都绑定方法，可以给class绑定方法

In [7]:
s2 = Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

In [8]:
def set_score(self, score):
    self.score = score
Student.set_score = set_score

In [9]:
s.set_score(100)
print(s.score)

100


In [10]:
s2.set_score(99)
print(s2.score)

99


### 使用_ _slots_ _

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性  
为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的_ _slots_ _变量，来限制该class实例能添加的属性  
使用_ _slots_ _要注意，_ _slots_ _定义的属性仅对当前类实例起作用，对继承的子类是不起作用的

In [11]:
class Student(object):
    __slots__ = ('name', 'age')

In [12]:
s = Student()
s.name = 'Micheal'
s.age = 25
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

In [13]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 99

## 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改

In [14]:
class Student(object):
    def get_score(self):
        return self._score
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value< 0 or value > 100:
            raise ValueError('score must between 0~100')
        self._score = value

In [15]:
s = Student()
s.set_score(60)
s.get_score()

60

In [16]:
s.set_score(10000)

ValueError: score must between 0~100

有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢  
Python内置的@property装饰器就是负责把一个方法变成属性调用的

In [17]:
class Student(object):
    @property
    def score(self):
        return self._score
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value < 0 or value > 100:
            raise ValueError('score must between 0~100')
        self._score = value

In [18]:
s = Student()
s.score = 60
print(s.score)
s.score = 9999

60


ValueError: score must between 0~100

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作  
还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性

In [19]:
class Student(object):
    @property
    def birth(self):
        return self._birth
    @birth.setter
    def birth(self, value):
        self._birth = value
    @property
    def age(self):
        return 2017 - self._birth

In [23]:
# _*_ coding: utf-8 _*_
class Screen(object):
    @property
    def width(self):
        return self._width
    @width.setter
    def width(self, value):
        self._width = value
    @property
    def height(self):
        return self._height
    @height.setter
    def height(self, value):
        self._height = value
    @property
    def resolution(self):
        return self._width * self._height

In [24]:
#测试
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过')
else:
    print('测试失败')

resolution = 786432
测试通过
